<a href="https://colab.research.google.com/github/nxxk23/AI-Engineer/blob/main/fraud/response.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import json as js
import re

In [3]:
hospital = pd.read_csv('/content/drive/MyDrive/AIEngineer/fraud/health_office.csv')
hospital.columns

<ipython-input-3-d69fde0a6e6e>:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  hospital = pd.read_csv('/content/drive/MyDrive/AIEngineer/fraud/health_office.csv')


Index(['ชื่อ', 'รหัส 9 หลักใหม่', 'รหัส 9 หลัก', 'รหัส 5 หลัก',
       'เลขอนุญาตให้ประกอบสถานบริการสุขภาพ 11 หลัก', 'ประเภทองค์กร',
       'ประเภทหน่วยบริการสุขภาพ', 'สังกัด', 'แผนก/กรม', 'ระดับโรงพยาบาล',
       'เตียงที่ใช้จริง', 'สถานะการใช้งาน', 'เขตบริการ', 'ที่อยู่',
       'รหัสจังหวัด', 'จังหวัด', 'รหัสอำเภอ', 'อำเภอ/เขต', 'รหัสตำบล',
       'ตำบล/แขวง', 'หมู่', 'รหัสไปรษณีย์', 'แม่ข่าย', 'วันที่ก่อตั้ง',
       'วันที่ปิดบริการ', 'อัพเดตล่าสุด(เริ่ม 05/09/2566)'],
      dtype='object')

In [4]:
data = pd.read_csv('/content/drive/MyDrive/AIEngineer/fraud/opd_discharge_August_2024.csv')
cols_to_drop = [col for col in data.columns if col.endswith('_txt')]
data = data.drop(columns=cols_to_drop)
df = data[data['claim_status'].isin(['Approve', 'Decline'])]
print(df['claim_status'].value_counts())

claim_status
Approve    15581
Decline      184
Name: count, dtype: int64


In [5]:
df['hospital_code'] = df['hospital_code'].astype(str)
hospital['hospital_code'] = hospital['รหัส 5 หลัก'].str.replace('="', '').str.replace('"', '')
hospital = hospital[['hospital_code', 'ชื่อ']].rename(columns={'ชื่อ': 'hospital_name'})
df = df.merge(hospital, on='hospital_code', how='left')

<ipython-input-5-83a06c097e2e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hospital_code'] = df['hospital_code'].astype(str)


In [6]:
hospital[hospital['hospital_code'] == '40765']

,hospital_code,hospital_name
1394,40765,โรงพยาบาลทั่วไปขนาดใหญ่เกษมราษฎร์ รัตนาธิเบศร์


In [7]:
focus = df[df['order_item'] != '[]'][['hospital_code', 'hospital_name', 'cid', 'insurance', 'claim_status','total_bill_amount', 'diagnosis', 'billing', 'order_item', 'response']]

In [8]:
decline = focus[focus['claim_status'].astype(str)=='Decline']
decline['insurance'].value_counts()

,count
insurance,
tli,23
aia,19
mtl,12
fwd,3


## **❗case: decline**
- **tli**: เข้าข้อยกเว้น, ICD10 อยู่ในกลุ่มโรค

- **aia**: ไม่สามารถเคลมได้ เนื่องจากสัญญาเพิ่มเติมสิ้นผลบังคับ, เบิกซ้ำซ้อนมากกว่า 1 ครั้ง/วัน, ต้องการข้อมูลทางการแพทย์เพิ่มเติม, ผลประโยชน์คุ้มครองเฉพาะค่ารักษาพยาบาลกรณีอุบัติเหตุ

- **mtl**: กรุณาสำรองจ่ายและส่งเอกสารการเคลมให้กับบริษัทฯโดยตรง, โรคมีระยะเวลาการรอคอย 180 วัน, ส่ง fax claim, เงื่อนไขโรคยกเว้นเฉพาะรายบุคคล, โรคที่เข้าข่ายในเงื่อนไขข้อยกเว้นกรมธรรม์สัญญาสุขภาพ

- **fwd**: copay == ผู้ป่วยต้องจ่ายเอง

In [9]:
js.loads(decline[decline['insurance']=='fwd']['response'].iloc[2])

{'detail': [{'benefit': [],
   'non_cover': '',
   'benefit_code': '',
   'copay_amount': '1792.00',
   'coverage_code': '',
   'excess_amount': '1792.00',
   'policy_number': 'C27363504',
   'order_sequence': '1',
   'trailer_number': '',
   'approved_amount': '0.00',
   'result_description': ''}],
 'claim_no': '202408062749',
 'insurer_detail': [{'claim_no': '202408062749',
   'non_cover': '',
   'benefit_code': '',
   'copay_amount': '1792.00',
   'coverage_code': '',
   'excess_amount': '1792.00',
   'policy_number': 'C27363504',
   'insurance_type': '',
   'order_sequence': '1',
   'trailer_number': '',
   'approved_amount': '0.00',
   'result_description': ''}],
 'transaction_no': '6d360234-c15a-4dbe-8c40-d123bb501458',
 'final_pss_score': '0',
 'transaction_uid': 'eb49a01c-39d3-4938-966e-9b382f3a3cb5',
 'total_excess_amount': '1792.00',
 'total_approved_amount': '0.00'}

In [10]:
approve = focus[focus['claim_status'].astype(str)=='Approve']
approve['insurance'].value_counts()

,count
insurance,
mtl,2068
aia,1572
fwd,868
tli,265


เพื่อค้นหา `order_item` ที่เป็นรายการที่พบร่วมกันบ่อยในผู้ป่วยที่เป็นโรคเดียวกัน **และหาค่าราคากลาง**ของ `order_item` นั้น

In [25]:
filtered_data = focus[
    focus['diagnosis'].apply(lambda x: any(d.get('icd10') == 'K300' for d in x))
]
filtered_data

,hospital_code,hospital_name,cid,insurance,claim_status,total_bill_amount,diagnosis,billing,order_item,response
868,12026,โรงพยาบาลเกษมราษฎร์ศรีบุรินทร์,8578573004852,mtl,Decline,2646.0,"[{'icd10': 'K300', 'dx_name': 'Functional Dysp...","[{""billing_initial"": ""160"", ""billing_discount""...","[{'initial': '160', 'item_id': 'N-737-1-03', '...","{""detail"": [{""benefit"": [{""simb_code"": ""1.1.12..."
1038,12026,โรงพยาบาลเกษมราษฎร์ศรีบุรินทร์,3579900207611,mtl,Approve,2706.0,"[{'icd10': 'K300', 'dx_name': 'Functional Dysp...","[{""billing_initial"": ""1922.00"", ""billing_disco...","[{'initial': '753', 'item_id': '0201A02P0030O0...","{""detail"": [{""benefit"": [{""simb_code"": ""1.1.1(..."
1317,41556,โรงพยาบาลเกษมราษฎร์ รามคำแหงโรงพยาบาลทั่วไปขนา...,1100400059409,mtl,Approve,965.0,"[{'icd10': 'K300', 'dx_name': 'Functional Dysp...","[{""billing_initial"": ""175"", ""billing_discount""...","[{'initial': '175', 'item_id': 'NS', 'discount...","{""detail"": [{""benefit"": [{""simb_code"": ""1.1.1(..."
2377,11558,โรงพยาบาลซีจีเอชโรงพยาบาลทั่วไปขนาดใหญ่,1100501349330,mtl,Approve,1374.0,"[{'icd10': 'K300', 'dx_name': 'Functional Dysp...","[{""billing_initial"": ""360.00"", ""billing_discou...","[{'initial': '190', 'item_id': 'DRUG', 'discou...","{""detail"": [{""benefit"": [{""simb_code"": ""1.1.1(..."
2407,11558,โรงพยาบาลซีจีเอชโรงพยาบาลทั่วไปขนาดใหญ่,3311000651527,mtl,Approve,1063.0,"[{'icd10': 'K291', 'dx_name': 'Other acute gas...","[{""billing_initial"": ""600"", ""billing_discount""...","[{'initial': '600', 'item_id': 'DFO', 'discoun...","{""detail"": [{""benefit"": [{""simb_code"": ""1.1.1(..."
6478,40765,โรงพยาบาลทั่วไปขนาดใหญ่เกษมราษฎร์ รัตนาธิเบศร์,2350190000019,mtl,Approve,2000.0,"[{'icd10': 'K300', 'dx_name': 'Functional Dysp...","[{""billing_initial"": ""1314.00"", ""billing_disco...","[{'initial': '521', 'item_id': '0101C06L00', '...","{""detail"": [{""benefit"": [{""simb_code"": ""1.1.1(..."
13039,41556,โรงพยาบาลเกษมราษฎร์ รามคำแหงโรงพยาบาลทั่วไปขนา...,1103701994801,mtl,Approve,1420.0,"[{'icd10': 'K300', 'dx_name': 'Functional Dysp...","[{""billing_initial"": ""800.00"", ""billing_discou...","[{'initial': '269', 'item_id': '0101A15O0020L0...","{""detail"": [{""benefit"": [{""simb_code"": ""1.1.1(..."
13329,11787,โรงพยาบาลเกษมราษฎร์อินเตอร์เนชั่นแนล รัตนาธิเบ...,1103701640316,mtl,Approve,3553.0,"[{'icd10': 'K300', 'dx_name': 'Functional Dysp...","[{""billing_initial"": ""100"", ""billing_discount""...","[{'initial': '100', 'item_id': 'SBV01', 'disco...","{""detail"": [{""benefit"": [{""simb_code"": ""1.1.1(..."


In [27]:
import json

# กรองข้อมูลที่เป็นโรคที่ต้องการ เช่น "K300"
filtered_data = focus[
    focus['diagnosis'].apply(lambda x: any(d.get('icd10') == 'K300' for d in x))
]

# สร้าง list เก็บ order_item ที่เกี่ยวข้องกับโรคที่กรองได้
order_items = []
for items in filtered_data['order_item']:
    for item in items:
        order_items.append(item)

# แปลง order_items เป็น DataFrame เพื่อช่วยในการคำนวณ
order_items_df = pd.DataFrame(order_items)

# แปลง initial เป็น float ใน order_items_df
order_items_df['initial'] = order_items_df['initial'].astype(float)

# คำนวณค่าราคากลาง (mean) ของแต่ละ item_id, รวม item_name และจำนวนการสั่ง (count)
order_item_summary = order_items_df.groupby(['item_id', 'item_name']).agg(
    average_initial_price=('initial', 'mean'),
    order_count=('initial', 'count')
).reset_index()

# แสดงผลรายการ order_item, item_name, ราคากลาง และจำนวนการสั่ง
order_item_summary

,item_id,item_name,average_initial_price,order_count
0,0101A08F0020L01,FAMOTIDINE 20 MG.TAB..,343.0,1
1,0101A15O0020L01,OMEPRAZOLE 20 MG CAP. (ก),269.0,1
2,0101A15O0020L01,OMEPRAZOLE 20 MG.CAP..,427.0,1
3,0101A15O0020L01,Omeprazole 20 mg (กส),338.0,1
4,0101B01A0080L01,SIMETHICONE TAB. 80 MG (ก),167.5,2
5,0101B05D00,DOMPERIDONE TAB.,630.0,1
6,0101B05D0010L01,DOMPERIDONE 10 MG TAB.,113.0,1
7,0101B05D0010L01,DOMPERIDONE 10 MG.TAB..,169.0,1
8,0101B10M0010L01,METOCLOPRAMIDE 10 MG.TAB,329.0,1
9,0101C05H0010L01,HY-OS-CINE 10 MG TAB.,107.0,1


In [12]:
import json

# แปลง JSON string ให้เป็น dictionary หรือ list
focus['diagnosis'] = focus['diagnosis'].apply(lambda x: json.loads(x))
focus['order_item'] = focus['order_item'].apply(lambda x: json.loads(x))

In [21]:
import json

# กรองข้อมูลที่เป็นโรคที่ต้องการ เช่น "K300"
filtered_data = focus[
    focus['diagnosis'].apply(lambda x: any(d.get('icd10') == 'J069' for d in x))
]

# สร้าง list เก็บ order_item ที่เกี่ยวข้องกับโรคที่กรองได้
order_items = []
for items in filtered_data['order_item']:
    for item in items:
        order_items.append(item)

# แปลง order_items เป็น DataFrame เพื่อช่วยในการคำนวณ
order_items_df = pd.DataFrame(order_items)

# แปลง initial เป็น float ใน order_items_df
order_items_df['initial'] = order_items_df['initial'].astype(float)

# คำนวณค่าราคากลาง (mean) ของแต่ละ item_id, รวม item_name และจำนวนการสั่ง (count)
order_item_summary = order_items_df.groupby(['item_id', 'item_name']).agg(
    average_initial_price=('initial', 'mean'),
    order_count=('initial', 'count')
).reset_index()

# แสดงผลรายการ order_item, item_name, ราคากลาง และจำนวนการสั่ง
order_item_summary[order_item_summary['item_id']=='SER']

,item_id,item_name,average_initial_price,order_count
270,SER,ค่าบริการพยาบาล,104.193548,93


In [22]:
import json

# กรองข้อมูลที่เป็นโรคที่ต้องการ เช่น "K300"
filtered_data = focus[
    focus['diagnosis'].apply(lambda x: any(d.get('icd10') == 'J029' for d in x))
]

# สร้าง list เก็บ order_item ที่เกี่ยวข้องกับโรคที่กรองได้
order_items = []
for items in filtered_data['order_item']:
    for item in items:
        order_items.append(item)

# แปลง order_items เป็น DataFrame เพื่อช่วยในการคำนวณ
order_items_df = pd.DataFrame(order_items)

# แปลง initial เป็น float ใน order_items_df
order_items_df['initial'] = order_items_df['initial'].astype(float)

# คำนวณค่าราคากลาง (mean) ของแต่ละ item_id, รวม item_name และจำนวนการสั่ง (count)
order_item_summary = order_items_df.groupby(['item_id', 'item_name']).agg(
    average_initial_price=('initial', 'mean'),
    order_count=('initial', 'count')
).reset_index()

# แสดงผลรายการ order_item, item_name, ราคากลาง และจำนวนการสั่ง
order_item_summary[order_item_summary['item_id']=='SER']

,item_id,item_name,average_initial_price,order_count
274,SER,ค่าบริการพยาบาล,100.0,13


In [23]:
import json

# กรองข้อมูลที่เป็นโรคที่ต้องการ เช่น "K300"
filtered_data = focus[
    focus['diagnosis'].apply(lambda x: any(d.get('icd10') == 'J000' for d in x))
]

# สร้าง list เก็บ order_item ที่เกี่ยวข้องกับโรคที่กรองได้
order_items = []
for items in filtered_data['order_item']:
    for item in items:
        order_items.append(item)

# แปลง order_items เป็น DataFrame เพื่อช่วยในการคำนวณ
order_items_df = pd.DataFrame(order_items)

# แปลง initial เป็น float ใน order_items_df
order_items_df['initial'] = order_items_df['initial'].astype(float)

# คำนวณค่าราคากลาง (mean) ของแต่ละ item_id, รวม item_name และจำนวนการสั่ง (count)
order_item_summary = order_items_df.groupby(['item_id', 'item_name']).agg(
    average_initial_price=('initial', 'mean'),
    order_count=('initial', 'count')
).reset_index()

# แสดงผลรายการ order_item, item_name, ราคากลาง และจำนวนการสั่ง
order_item_summary[order_item_summary['item_id']=='SER']

,item_id,item_name,average_initial_price,order_count
231,SER,ค่าบริการพยาบาล,114.285714,21
